## Legenda and Project Description.
<br> This is the notebook I used to do export the data from the google spreadsheet of the client, clean it and manipulate the data to calculate the averages and the "TRAC_Index" (average of all sections on which a company has been assessed). 
Once calculated all scores, I pushed again the new dataframes as tabs in the google spreadsheet where the raw data live. 
Once this was done I moved all this code in a app.py and utils.py file so that in the future the client can simply use the command line to create the new tables. 

In [21]:
# Imports 
import pandas as pd
import gspread
import numpy as np
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

In [2]:
# Access the Google Spreadsheet. 
# Make sure the file 'Jupyter_meets_GSheet-a279ad757691.json' is in the same folder as this Notebook. 
# Credits: www.countingcalculi.com/explanations/google_sheets_and_jupyter_notebooks/

scope = ['https://spreadsheets.google.com/feeds']

# Store credentials in variable we will then pass to the authorize function. 
credentials = ServiceAccountCredentials.from_json_keyfile_name('Jupyter_meets_GSheet-a279ad757691.json', scope)

# Store authorized Google client in a variable 
google_client = gspread.authorize(credentials)

# This is the key copied from the url of the Google Spreadsheet, see here: https://docs.google.com/spreadsheets/d/1IQ9BUHUCkc5bdgPYEPzUAMGaMjMQ2cOT9j2gtr4PnKw/edit#gid=1680607748
spreadsheet_key = '1ANV9TXjL75vUaxxuBCvMqYgc7fgg3uruUy_BdKKyr30'

# Allow the google client to open the spreadsheet using the spreadsheet_key, store the spreadsheet in book. 
book = google_client.open_by_key(spreadsheet_key)

# Which tab do we want to open?
worksheet = book.worksheet("Full_DB") 

# Convert table data into a dataframe
table = worksheet.get_all_values()

# Create and populate the questions DataFrame
This DataFrame will have  35 rows (companies) × 67 columns (questions). Questions can be answered using different scales. Here are the options: 
<br>OPTION 1: 0, 1: Binary (Yes or No questions). 
<br>OPTION 2: 0, 2: Binary (Yes or No questions, with the Yes having more weight than the Yes of OPTION 1).
<br>OPTION 2: 0, 1, 2: Categorical. 
<br>OPTION 3: 0, 1, 2, 3: Categorical.

NB: Question scores have not been normalised before the calculation of the average score per section.



In [4]:
# Create original DataFrame and store it in the folder. 
orig_df = pd.DataFrame(table[1:], columns=table[0]).fillna(value=np.nan) #fill missing values with NaN
orig_df.to_csv("raw_df")
copy_df = orig_df
# copy_df # It works! Hurray! Now hash it otherwise it'll keep popping up. 

# Create and populate the scores DataFrame

Given that the initial spreadsheet had been built using a naming convention that goes:

Feature_SectionNumber_Question_Number 

it was prefferred to use string manipulation to slice the dataframe, rather than use multi-indexing. 

In [5]:
# Create empty dictionary to map every section title to the number of questions they contain
sections_dict = {}

# Add all section names as key and a list of the questions they contain as list 
sections_dict["1"] = [1, 2, 3, 4, 5]
sections_dict["2"] = [1, 2, 3, 4, 5, 6, 7, 8, 9]
sections_dict["3"] = [1, 2, 3, 4, 5, 6, 7, 8, 9]
sections_dict["4"] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
sections_dict["5"] = [1, 2, 3, 4, 5, 6]
sections_dict["6"] = [1, 2, 3, 4, 5, 6]
sections_dict["7"] = [1, 2, 3, 4]
sections_dict["8"] = [1, 2, 3, 4, 5]
sections_dict["9"] = [1, 2, 3, 4, 5, 6, 7, 8]
sections_dict["10"] = [1, 2, 3, 4]

# List of all question numbers
questions_list = list(sections_dict.values())

# Create function to create the list of columns to keep from the copy DataFrame. Takes a string as an argument and 
# iteratively creates the names of the columns to keep extracting the number of the section and the the questions
# from the sections_dict object. 

def columns_to_keep (column_str = "name"):
    columns_to_keep_ls = []
    for key_i, value_ls in sections_dict.items():
        for question_i in range(len(value_ls)):
            columns_to_keep_ls.append("{}_{}_{}".format(column_str, key_i, value_ls[question_i]))
    return (columns_to_keep_ls)

# Create Scores_sectionNumber_questionNumber strings and add names and sectors columns to columns_to_keep variable.  
cols_ls = columns_to_keep("Score")
cols_ls.extend(("Company_Name", "Company_Sector"))

# Create the scores DataFrame, index is name of company
scores_df = copy_df[cols_ls].set_index("Company_Name")

# Push DataFrame to Google Spreadsheet (hashed because we don't want to mess up with the spreadsheet the client is using)
worksheet_name = 'Scores_Only'
d2g.upload(scores_df, spreadsheet_key, worksheet_name, credentials=credentials, row_names=True)

# For show
scores_df

,Score_1_1,Score_1_2,Score_1_3,Score_1_4,Score_1_5,Score_2_1,Score_2_2,Score_2_3,Score_2_4,Score_2_5,...,Score_9_4,Score_9_5,Score_9_6,Score_9_7,Score_9_8,Score_10_1,Score_10_2,Score_10_3,Score_10_4,Company_Sector
Company_Name,,,,,,,,,,,,,,,,,,,,,
Abb Italia,2,2,2,2,2,2,2,1,2,2,...,1,2,2,2,2,2,2,0,0,Meccanica
Ali,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Meccanica
Armani,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fashion
Barilla Holding,0,2,2,2,2,2,2,0,2,2,...,0,0,2,0,2,0,0,0,0,Food
Brembo,1,2,0,2,2,2,2,1,2,2,...,1,2,0,2,2,2,2,0,0,Meccanica
Calzedonia,0,2,0,0,1,2,2,1,2,2,...,0,0,0,0,2,0,0,0,0,Fashion
Cassa Depositi e Prestiti,0,2,0,2,2,2,2,1,2,2,...,1,0,0,2,2,0,0,0,0,Finanziario
Costa crociere,1,2,2,2,2,2,2,1,2,2,...,1,2,2,2,0,0,0,0,0,Top brand
Danieli & C.,0,2,0,0,1,2,1,1,2,2,...,1,0,0,0,0,0,0,0,0,Meccanica


# Calculate all section averages and assign them to bands

<br> Every section has a different amount of maximum points that can be obtained so I created the variable max_scores_ls variable for the calculation of the averages. 
<br> The bands associated to the ranges of possible scores are as follows:
<br> 0.000 - 0.250 : Non Soddisfacente (*Unsatisfactory*)
<br> 0.251 - 0.500 : Poco Soddisfacente (*Barely Satisfactory*)
<br> 0.501 - 0.750 : Soddisfacente (*Satisfactory*)
<br> 0.751 - 1.000 : Eccellente (*Excellent*)

In [7]:
# Make list of sections max_score possible
max_scores_ls = [10, 17, 18, 20, 12, 12, 10, 10, 16, 8]

# Create a function that takes the name of a company and the section over which to calculate the score
# It should output a float as the score of t
def calculate_section_scores(company_name="name", section=0):
    ls = (
        scores_df.loc["{}".format(company_name)]
        # Hard code the '1' as I will always want to start from the 1st question.
        # Get length of list in questions_list corresponding to last question in that section.
        ["Score_{}_1".format(section):"Score_{}_{}".format(section, len(questions_list[section-1]))]
    
    ).values[:]
    
    score_flt = (pd.to_numeric(ls).sum())/max_scores_ls[section-1]
    
    return (score_flt)

# Sanity check, treats NaN correctly? Yes. 
# calculate_section_scores(company_name="Brembo", section=7)

# Create new dataframe to store results and averages of each company and each section.
col = ["Section_1", "Section_2", "Section_3", "Section_4", "Section_5", "Section_6", "Section_7", "Section_8", "Section_9", "Section_10", "TRAC_Index"]
sections_scores_df = pd.DataFrame(data=np.NaN, index = scores_df.index, columns = col)
sections_scores_df.loc["Averages", "Bands"] = np.NaN

# Create loop that stores all tuples in a new df, indexed on company_names.
# IMPROVEMENT: I tried to iteratively create a column (scores_df['Section_{}'.format(number)) and fill it in the loops but adding columns to the existing 
# I get a warning, so I manually created a new empty one and filled it in. 
for element in scores_df.index:
    for number in range(1,11):
        sections_scores_df.at[element, 'Section_{}'.format(number)] = round((calculate_section_scores(company_name=element, section=number)), 2)
    sections_scores_df.at[element, "TRAC_Index"] = round(sections_scores_df.loc[element].mean(), 2)
        
# Create loop that calculates average of all columns to get comparable results
for element in sections_scores_df.columns:
    sections_scores_df.at["Averages", element] = round(sections_scores_df[element].mean(), 2)

# Create dictionary to associate bands to score range. 
bands_dict = {0 : "Non Soddisfacente", 0.25 : "Poco Soddisfacente", 0.50 : "Soddisfacente", 0.75 : "Eccellente", 1.0 : "Whatevs"}

# Loop over every company 
for company in sections_scores_df.index:
    
    # Handles 0.00 exeption, as it will not be true that 0.00 > 0 in the following loop. 
    if sections_scores_df.at[company, "TRAC_Index"] == 0:
        sections_scores_df.loc[company, "Bands"] = list(bands_dict.values())[0]
        
    # Loop backwards from the highest possible band band 
    for i in reversed((range(0,5))): 
        
        # If the score between the lower bound of the band (included) and the upper bound (excluded) then attribute that band
        if  (
            sections_scores_df.at[company, "TRAC_Index"] < list(bands_dict.keys())[i] and
            sections_scores_df.at[company, "TRAC_Index"] >= list(bands_dict.keys())[i-1]
            ):
            sections_scores_df.loc[company, "Bands"] = list(bands_dict.values())[i-1]
            
# Add columns on sector by merging initial copy DataFrame with the section scores DataFrame on Company Name. 
sections_scores_df = pd.merge(sections_scores_df,copy_df[['Company_Name','Company_Sector']],on='Company_Name', how='left')
sections_scores_df = sections_scores_df.set_index("Company_Name")

# Push dataframe back to Google Spreadsheet 
# (hashed because we don't want to mess up with the spreadsheet the client is using)
worksheet_name = 'Scores_Data'
d2g.upload(sections_scores_df, spreadsheet_key, worksheet_name, credentials=credentials, row_names=True)

# For show
sections_scores_df

,Section_1,Section_2,Section_3,Section_4,Section_5,Section_6,Section_7,Section_8,Section_9,Section_10,TRAC_Index,Bands,Company_Sector
Company_Name,,,,,,,,,,,,,
Abb Italia,1.00,0.59,0.83,0.75,0.00,0.83,0.40,0.2,0.69,0.50,0.58,Soddisfacente,Meccanica
Ali,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,Non Soddisfacente,Meccanica
Armani,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,Non Soddisfacente,Fashion
Barilla Holding,0.80,0.76,0.78,0.45,0.50,0.75,0.00,0.6,0.38,0.00,0.50,Soddisfacente,Food
Brembo,0.70,0.88,0.94,0.65,0.50,0.00,NaN,0.5,0.56,0.50,0.58,Soddisfacente,Meccanica
Calzedonia,0.30,0.76,0.44,0.40,0.00,0.25,0.00,0.0,0.19,0.00,0.23,Non Soddisfacente,Fashion
Cassa Depositi e Prestiti,0.60,0.94,0.72,0.35,NaN,0.83,NaN,0.5,0.44,0.00,0.55,Soddisfacente,Finanziario
Costa crociere,0.90,0.65,0.56,0.80,0.33,0.75,0.50,0.2,0.56,0.00,0.53,Soddisfacente,Top brand
Danieli & C.,0.30,0.82,0.61,0.50,0.00,0.42,0.30,0.9,0.19,0.00,0.40,Poco Soddisfacente,Meccanica


# Calculate all sectors store results in new dataframe 


In [9]:
grouped_scores_df = pd.DataFrame()

for section_i in list(range(1, 11)):  
    grouped_scores_df = grouped_scores_df.append(round(sections_scores_df.groupby("Company_Sector")["Section_{}".format(section_i)].mean(), 2))

grouped_scores_df_transposed = grouped_scores_df.transpose()

# Push dataframe back to Google Spreadsheet 
# (hashed because we don't want to mess up with the spreadsheet the client is using)

worksheet_name = 'Sector_Averages'
d2g.upload(grouped_scores_df_transposed, spreadsheet_key, worksheet_name, credentials=credentials, row_names=True)

# For show
grouped_scores_df_transposed


,Section_1,Section_2,Section_3,Section_4,Section_5,Section_6,Section_7,Section_8,Section_9,Section_10
Altri settori,0.76,0.75,0.72,0.75,0.47,0.73,0.30,0.62,0.60,0.75
Calcio,0.42,0.63,0.57,0.30,0.17,0.58,0.10,0.04,0.15,0.10
Energia,0.92,0.94,0.84,0.71,0.45,0.73,NaN,0.56,0.74,0.80
Fashion,0.44,0.67,0.54,0.30,0.27,0.47,0.20,0.20,0.28,0.25
Finanziario,0.76,0.88,0.77,0.58,0.42,0.75,0.30,0.60,0.55,0.65
Food,0.54,0.53,0.53,0.34,0.30,0.43,0.00,0.34,0.32,0.10
Meccanica,0.54,0.65,0.62,0.49,0.27,0.33,0.38,0.36,0.38,0.20
Partecipate dallo Stato,0.74,0.92,0.71,0.69,0.42,0.80,0.00,0.38,0.56,0.20
Telefonia,0.74,0.90,0.82,0.59,0.58,0.76,0.10,0.64,0.80,0.70
Top brand,0.54,0.58,0.66,0.58,0.44,0.61,0.15,0.30,0.37,0.10


In [17]:
a = []
SECTIONS_NAMES_IT = {"Section_1":"Dichiarazioni \nPubbliche", 
                 "Section_2":"Modello \nAnticorruzione", 
                 "Section_3":"Codice",
                 "Section_4":"Whistleblowing", 
                 "Section_5":"Lobbying", 
                 "Section_6":"Conflitto \nd'Interesse",
                 "Section_7":"Finanziamenti \nPolitici",
                 "Section_8":"Traspareza \nOrganizzativa",
                 "Section_9":"Formazione", 
                 "Section_10":"Sostenibilità"}


In [23]:
df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))
df

,A,B,C,D
0,88,1,82,63
1,72,31,21,59
2,38,17,94,87
3,91,88,94,82
4,85,53,31,79
5,6,71,83,19
6,45,77,68,71
7,98,17,78,32
8,58,17,99,98
9,49,95,77,69


In [26]:
SECTIONS_DICT = {
                 "1" : [1, 2, 3, 4, 5], 
                 "2" : [1, 2, 3, 4, 5, 6, 7, 8, 9],
                 "3" : [1, 2, 3, 4, 5, 6, 7, 8, 9],
                 "4" : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                 "5" : [1, 2, 3, 4, 5, 6],
                 "6" : [1, 2, 3, 4, 5, 6],
                 "7" : [1, 2, 3, 4],
                 "8" : [1, 2, 3, 4, 5],
                 "9" : [1, 2, 3, 4, 5, 6, 7, 8],
                 "10": [1, 2, 3, 4]
                }
# List of all question numbers
QUESTIONS_LS = list(SECTIONS_DICT.values())

In [29]:
len(QUESTIONS_LS[1])

9